### Imports and configuration

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional

from run_experiments import run_qse_experiment, run_pqse_experiment

In [ ]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pd.set_option("display.max_columns", None)

# Apply the default theme
sns.set_theme(style="ticks")
sns.despine()

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "sans-serif",
        "font.sans-serif": ["Helvetica"],
        "font.size": "60",
    }
)

plt.rc("text.latex", preamble=r"\usepackage{braket}\usepackage{amsmath}")

### Experiment setup functions. 

In [ ]:
def setup_qse_experiment(
    noise_str, min_noise_seed, max_noise_seed, min_kbo_final, max_kbo_final
):
    frames = []

    for noise_seed in range(min_noise_seed, max_noise_seed + 1):
        print("Running QSE with noise seed: ", noise_seed)

        for kbo_final in range(min_kbo_final, max_kbo_final + 1):
            experiment_params = {
                "seed": 2,
                "kbo_final": kbo_final,  # final Krylov basis order
                "ref_st_name": "J_zero",  # reference state name
                "noise_type": "finite_shot_noise",
                "noise_str": noise_str,  # noise strength
                "n_seed": noise_seed,
                "thresh": False,  # thresholding
                # problem specific params
                "s_s": 10,  # size state
                "J": 0.1,  # inter spin coupling
                "h_bnd": 1,  # bound on magnitude of on site disorder
            }

            frames.append(run_qse_experiment(experiment_params))

    return pd.concat(frames)

In [ ]:
def setup_tqse_experiment(
    noise_str,
    min_noise_seed,
    max_noise_seed,
    min_kbo_final,
    max_kbo_final,
    threshold_scales,
):
    frames = []

    for scale in threshold_scales:
        for noise_seed in range(min_noise_seed, max_noise_seed + 1):
            print(
                "Running TQSE with scaling factor {} & noise seed {}".format(
                    scale, noise_seed
                )
            )

            for kbo_final in range(min_kbo_final, max_kbo_final + 1):
                experiment_params = {
                    "seed": 2,
                    "kbo_final": kbo_final,  # final Krylov basis order
                    "ref_st_name": "J_zero",  # reference state name
                    "noise_type": "finite_shot_noise",
                    "noise_str": noise_str,  # noise strength
                    "n_seed": noise_seed,
                    "thresh": True,  # thresholding,
                    "thresh_scale": scale,
                    # problem specific params
                    "s_s": 10,  # size state
                    "J": 0.1,  # inter spin coupling
                    "h_bnd": 1,  # bound on magnitude of on site disorder
                }

                frames.append(run_qse_experiment(experiment_params))

    return pd.concat(frames)

In [ ]:
def setup_pqse_experiment(
    noise_str, min_noise_seed, max_noise_seed, min_kbo_final, max_kbo_final
):
    frames = []

    for noise_seed in range(min_noise_seed, max_noise_seed + 1):
        print("Running PQSE with noise seed: ", noise_seed)
        for kbo_final in range(min_kbo_final, max_kbo_final + 1):
            # experiment parameters
            experiment_params = {
                "seed": 2,
                "ref_upd": True,  # reference update
                "kbo_final": kbo_final,  # final Krylov basis order
                "s_s": 10,  # size state
                "ref_st_name": "J_zero",  # reference state name
                "noise_type": "finite_shot_noise",
                "noise_str": noise_str,  # noise strength
                "n_seed": noise_seed,
                "thold": "None",
                "J": 0.1,  # inter spin coupling
                "h_bnd": 1,  # bound on magnitude of on site disorder
            }

            frames.append(run_pqse_experiment(experiment_params))

    return pd.concat(frames)

### Dataframe functions

In [ ]:
def get_df_averages(
    df: pd.DataFrame,
    drop_columns: Optional[list] = None,
    group_columns: Optional[list] = None,
):
    if drop_columns is None and group_columns is None:
        return df

    if drop_columns is not None:
        aggregated_df = df.drop(drop_columns, axis=1)

    if group_columns is not None:
        aggregated_df = aggregated_df.groupby(group_columns).aggregate(["mean", "sem"])

    aggregated_df = aggregated_df.reset_index()

    return aggregated_df

### Setup and run experiments

In [ ]:
noise_str = 1e-6
min_noise_seed = 0
max_noise_seed = 199
min_kbo_final = 1
max_kbo_final = 30
# choose known good and bad threshold scaling factors 
# error minimisation over these factors is done automatically, so feel free to add more. 
threshold_scales = [0.379, 3] 

In [ ]:
qse_df = setup_qse_experiment(
    noise_str, min_noise_seed, max_noise_seed, min_kbo_final, max_kbo_final
)

In [ ]:
tqse_df = setup_tqse_experiment(
    noise_str,
    min_noise_seed,
    max_noise_seed,
    min_kbo_final,
    max_kbo_final,
    threshold_scales,
)

In [ ]:
pqse_df = setup_pqse_experiment(
    noise_str, min_noise_seed, max_noise_seed, min_kbo_final, max_kbo_final
)

### Process data

In [ ]:
ave_qse_df = get_df_averages(
    qse_df,
    drop_columns=["noise_type", "init_reference_state"],
    group_columns=["noise_strength", "kbo_final"],
)

ave_tqse_df = get_df_averages(
    tqse_df,
    drop_columns=["noise_type", "init_reference_state"],
    group_columns=["noise_strength", "kbo_final", "thresholding_scale"],
)


ave_pqse_df = get_df_averages(
    pqse_df,
    drop_columns=["noise_type", "init_reference_state"],
    group_columns=["noise_strength", "reference_update", "kbo_final"],
)

Get threshold scaling factor associated with minimum energy

In [ ]:
min_error_tqse_df = ave_tqse_df.groupby("thresholding_scale").min()
min_energy_error = min(min_error_tqse_df["energy_err_rel"]["mean"])
min_error_threshold_scale = min_error_tqse_df[
    min_error_tqse_df[("energy_err_rel", "mean")] == min_energy_error
].reset_index()["thresholding_scale"][0]
min_error_threshold_scale

### Plot results

In [ ]:
import warnings

warnings.filterwarnings("ignore")

markers = ["D", "X", "o"]

colors = [_ for (idx, _) in enumerate(sns.color_palette("colorblind")) if idx != 1][:3]
colors = [colors[2], colors[0], colors[1]]

kwargs = {"edgecolor": "k", "s": 30}  # for edge color

# QSE
qse_scatter_plot = sns.scatterplot(
    data=ave_qse_df[ave_qse_df["noise_strength"] == noise_str],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    marker=markers[0],
    color=colors[0],
    **kwargs,
    label="QSE",
    legend=None
)


sns.lineplot(
    data=ave_qse_df[ave_qse_df["noise_strength"] == noise_str],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    estimator=None,
    ls=":",
    color=colors[0],
    alpha=0.5,
    legend=None,
    markers=False,
)

plt.errorbar(
    ave_qse_df[ave_qse_df["noise_strength"] == noise_str]["kbo_final"],
    ave_qse_df[ave_qse_df["noise_strength"] == noise_str]["energy_err_rel"][
        "mean"
    ],
    ave_qse_df[ave_qse_df["noise_strength"] == noise_str]["energy_err_rel"][
        "sem"
    ],
    capsize=5,
    ecolor="k",
    marker="none",
    ls="None",
)


# TQSE
sns.lineplot(
    data=ave_tqse_df[
        (ave_tqse_df["thresholding_scale"] == min_error_threshold_scale)
        & (ave_tqse_df["noise_strength"] == noise_str)
    ],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    estimator=None,
    color=colors[1],
    ls=":",
    alpha=0.5,
    legend=None,
)


thqse_scatter_plot = sns.scatterplot(
    data=ave_tqse_df[
        (ave_tqse_df["thresholding_scale"] == min_error_threshold_scale)
        & (ave_tqse_df["noise_strength"] == noise_str)
    ],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    marker=markers[1],
    color=colors[1],
    **kwargs,
    label="TQSE",
    legend=None
)

plt.errorbar(
    ave_tqse_df[
        (ave_tqse_df["thresholding_scale"] == min_error_threshold_scale)
        & (ave_tqse_df["noise_strength"] == noise_str)
    ]["kbo_final"],
    ave_tqse_df[
        (ave_tqse_df["thresholding_scale"] == min_error_threshold_scale)
        & (ave_tqse_df["noise_strength"] == noise_str)
    ]["energy_err_rel"]["mean"],
    ave_tqse_df[
        (ave_tqse_df["thresholding_scale"] == min_error_threshold_scale)
        & (ave_tqse_df["noise_strength"] == noise_str)
    ]["energy_err_rel"]["sem"],
    capsize=5,
    ecolor="k",
    marker="none",
    ls="None",
)

# PQSE
sns.lineplot(
    data=ave_pqse_df[ave_pqse_df["noise_strength"] == noise_str],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    estimator=None,
    color=colors[2],
    ls=":",
    alpha=0.5,
    legend=None,
)

sns.scatterplot(
    data=ave_pqse_df[ave_pqse_df["noise_strength"] == noise_str],
    x="kbo_final",
    y=("energy_err_rel", "mean"),
    marker=markers[2],
    color=colors[2],
    **kwargs,
    label="PQSE",
    legend=None
)

plt.errorbar(
    ave_pqse_df[ave_pqse_df["noise_strength"] == noise_str]["kbo_final"],
    ave_pqse_df[ave_pqse_df["noise_strength"] == noise_str]["energy_err_rel"][
        "mean"
    ],
    ave_pqse_df[ave_pqse_df["noise_strength"] == noise_str]["energy_err_rel"][
        "sem"
    ],
    capsize=5,
    ecolor="k",
    marker="none",
    ls="None",
)


qse_scatter_plot.set(ylabel=r"$\overline{\epsilon_{\textrm{rel}}}$", xlabel=r"$R$")
qse_scatter_plot.set(yscale="log")

plt.legend(fontsize=15, bbox_to_anchor=(1, 1))
plt.tick_params(which="both", direction="in")

plt.annotate(
    r"$\delta =$ {}".format(noise_str),
    (425, 200),
    xycoords="figure points",
    fontsize=15,
)

plt.show()